In [3]:
## load packages
import logging
import matplotlib.pyplot as plt
import numpy as np
import os, os.path
import pandas as pd
import temp as tmp
import sisepuede as ssp
import sqlalchemy
import sql_utilities as sqlutil
import support_functions as sf
from typing import *
import warnings
warnings.filterwarnings("ignore")


# Initialize SISEPUEDE 
- setup with whatever regions you want
- will initialize Julia, but you don't have to run with it


In [4]:
region = "colombia"
sisepuede = ssp.SISEPUEDE(
    "calibrated",
    regions = [region],
)



2023-07-17 13:12:17,519 - INFO - Successfully initialized SISEPUEDEFileStructure.
2023-07-17 13:12:17,521 - WARNING - Missing key dict_dimensional_keys: key time_series not found. Tables that rely on the time_series will not have index checking.
2023-07-17 13:12:17,522 - INFO - 	Setting export engine to 'sqlite'.
2023-07-17 13:12:17,542 - WARNING - No index fields defined. Index field values will not be checked when writing to tables.
2023-07-17 13:12:17,543 - INFO - Successfully instantiated table ANALYSIS_METADATA
2023-07-17 13:12:17,545 - WARNING - No index fields found in ATTRIBUTE_DESIGN. Initializing index fields.
2023-07-17 13:12:17,545 - INFO - Successfully instantiated table ATTRIBUTE_DESIGN
2023-07-17 13:12:17,547 - WARNING - No index fields found in ATTRIBUTE_LHC_SAMPLES_EXOGENOUS_UNCERTAINTIES. Initializing index fields.
2023-07-17 13:12:17,548 - INFO - Successfully instantiated table ATTRIBUTE_LHC_SAMPLES_EXOGENOUS_UNCERTAINTIES
2023-07-17 13:12:17,550 - WARNING - No index

# Build an input dataframe
- input dataframes are generated on the fly
- use FutureTrajectories.generate_future_from_lhs_vector() to generate
- Below, see ``?ft.generate_future_from_lhs_vector`` for arguments
- Data frame produces inputs for each strategy_id defined in the selected region and the specified future_id?

In [5]:
# build an input data frame (baseline future)

ft = (
    sisepuede.experimental_manager
    .dict_future_trajectories
    .get(region)
)
df_in = ft.generate_future_from_lhs_vector(None, None, 0, True)



# Get a strategy dataframe to play with
- try modifying this data frame (``df_eval``) directly to verify behavior


In [7]:
strat = 0

# NOTE: SISEPUEDEMOdels.project requires that scenario dimensions are stripped, so drop them
df_eval = (
    df_in[
        df_in[sisepuede.key_strategy].isin([strat])
    ]
    .drop([sisepuede.key_strategy, sisepuede.key_future], axis = 1)
)

# Modify inputs directly, then project using SISEPUEDEModels
- if you're not evaluating fuel production outputs, you can set ``include_electricity_in_energy = False`` to avoid running iit

In [13]:
df_out = sisepuede.models.project(
    df_eval,
    include_electricity_in_energy = False, # set to true to run NemoMod
)

2023-07-17 13:18:36,469 - INFO - Running AFOLU model
2023-07-17 13:18:36,872 - INFO - AFOLU model run successfully completed
2023-07-17 13:18:36,872 - INFO - Running CircularEconomy model
2023-07-17 13:18:36,994 - INFO - CircularEconomy model run successfully completed
2023-07-17 13:18:36,995 - INFO - Running IPPU model
2023-07-17 13:18:37,184 - INFO - IPPU model run successfully completed
2023-07-17 13:18:37,185 - INFO - Running Energy model (NonElectricEnergy without Fugitive Emissions)
2023-07-17 13:18:37,236 - DEBUG - Missing elasticity information found in 'project_energy_consumption_by_fuel_from_effvars': using specified future demands.
2023-07-17 13:18:37,494 - INFO - NonElectricEnergy without Fugitive Emissions model run successfully completed
2023-07-17 13:18:37,494 - INFO - Running Energy (Fugitive Emissions)
2023-07-17 13:18:37,536 - WARNING - Warning in project_enfu_production_and_demands: Variable 'Energy Demand by Fuel in Energy Technology' not found in the data frame. It

In [14]:
?sisepuede.models.project

Signature:
sisepuede.models.project(
    df_input_data: pandas.core.frame.DataFrame,
    check_results: bool = True,
    fields_check: Union[List[str], str, NoneType] = 'emissions_output_subsector_aggregate',
    include_electricity_in_energy: bool = True,
    models_run: Optional[List[str]] = None,
    regions: Union[List[str], str, NoneType] = None,
    run_integrated: bool = True,
    time_periods_run: Optional[List[int]] = None,
    **kwargs,
) -> pandas.core.frame.DataFrame
Docstring:
Execute the SISEPUEDE DAG.

Function Arguments
------------------
df_input_data: DataFrame containing SISEPUEDE inputs

Optional Arguments
------------------
- models_run: list of sector models to run as defined in
        SISEPUEDEModels.model_attributes. Can include the following values:

        * AFOLU (or af)
        * Circular Economy (or ce)
        * IPPU (or ip)
        * Energy (or en)
                * Note: set include_electricity_in_energy = False to avoid
                        running

# NOTE: You can access variables using their names (E.g., tables  sisepuede.readthedocs.io )
- E.g. access forest sequestration using the ``"Forest Sequestration Emission Factor"`` variable
- Some variables have math terms in their names, and it can be easier to access using the modvar_SUBSEC variable within the model. 
    - E.g., ``sisepuede.models.model_afolu.modvar_frst_sq_co2`` stores this variable

In [22]:
# verify your inputs
sisepuede.model_attributes.get_standard_variables(
    df_eval,
    sisepuede.models.model_afolu.modvar_frst_sq_co2,
    include_time_period = True,
).tail()

,time_period,ef_frst_sequestration_mangroves_kt_co2_ha,ef_frst_sequestration_primary_kt_co2_ha,ef_frst_sequestration_secondary_kt_co2_ha
31,31,0.005408,0.001267,0.006885
32,32,0.005408,0.001267,0.006885
33,33,0.005408,0.001267,0.006885
34,34,0.005408,0.001267,0.006885
35,35,0.005408,0.001267,0.006885


In [23]:
# look at some outputs
sisepuede.model_attributes.get_standard_variables(
    df_out,
    sisepuede.models.model_afolu.modvar_frst_emissions_co2_sequestration,
    include_time_period = True,
).tail()


,time_period,emission_co2e_co2_frst_sequestration_mangroves,emission_co2e_co2_frst_sequestration_primary,emission_co2e_co2_frst_sequestration_secondary
31,31,-5.374243,-32.682206,-3.570667
32,32,-5.287318,-30.919624,-3.483691
33,33,-5.181310,-29.214807,-3.391254
34,34,-5.055787,-27.568753,-3.293412
35,35,-4.910366,-25.982192,-3.190182
